# PyG explainability

NOTE: many explainability finctionality in PyG documentation have not been yet implemented 

In [ ]:
from utils.models import model_selector
from utils.datasets import load_dataset, parse_config
#from explainers.PGExplainer import PGExplainer

import torch
from torch_geometric.data import Data
from torch_geometric.explain import Explainer, GNNExplainer

import os
GNN_MODEL = "GNN"
DATASET = "bashapes"
cfg_path = f"./configs/{GNN_MODEL}/{DATASET}.json"
cfg = parse_config(config_path=cfg_path)

In [ ]:
# A homogenous graph data object.
# data = Data(...)
dataset, _ = load_dataset(dataset="syn1")
data = dataset[0]

# instantiate GNN model
model, _ = model_selector(paper=GNN_MODEL, dataset="syn1", pretrained=False, config=cfg)

## ZAVVE: i'm using pyg '2.2.0', that is not the 'latest' version,
# in the 'latest' version the definition of ExplainerConfig and
# ModelConfig have been changed;
explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=200),
    explainer_config=dict(
        explanation_type='model',
        node_mask_type='attributes',
        edge_mask_type='object',
    ),
    model_config=dict(
        mode='classification',
        task_level='node',
        return_type='log_probs',  # Model returns log of probability.
    ),
)

# Generate explanation for the node at index `10`:
explanation = explainer(data.x, data.edge_index, index=450)

print("\nexplanation...")
print("edge_mask:", explanation.edge_mask)
print("expl_size (#edges):", torch.sum(explanation.edge_mask >= 0.5))
#print(explanation.node_mask)   # only (maybe) in latest version of PyG
#explanation.visualize_graph()  # 